# Preprocessing

Preprocessing notebook

In [6]:
import numpy as np
import pandas as pd

pd.options.plotting.backend = "plotly"

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_ipcw
from sksurv.util import Surv

df = pd.read_csv("../../data/clinical_train.csv")
df_eval = pd.read_csv("../../data/clinical_val.csv")
maf_df = pd.read_csv("../../data/molecular_train.csv")
maf_eval = pd.read_csv("../../data/molecular_val.csv")
target_df = pd.read_csv("../../data/target_train.csv")

In [7]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ======================================================================
# 0. Data preparation
# ======================================================================

df_noid = df.drop(columns=["ID", "CENTER"], errors="ignore")
maf_noid = maf_df.drop(columns=["ID"], errors="ignore")


# ======================================================================
# 1. Missing values - Clinical
# ======================================================================

missing_clinical = df_noid.isnull().sum()
missing_clinical_pct = (missing_clinical / len(df_noid) * 100).sort_values()

fig1 = go.Figure(
    go.Bar(
        x=missing_clinical_pct.values,
        y=missing_clinical_pct.index,
        orientation="h",
        text=[f"{v:.1f}%" for v in missing_clinical_pct.values],
        textposition="outside"
    )
)

fig1.update_layout(
    xaxis_title="Percentage of Missing Values (%)",
    yaxis_title="Features",
    margin=dict(l=120, r=40, t=10, b=40),
    showlegend=False
)

fig1.update_xaxes(showgrid=False)
fig1.update_yaxes(showgrid=False)

fig1.show()


# ======================================================================
# 2. Missing heatmap - Clinical
# ======================================================================

missing_matrix = df_noid.isnull().astype(int)

sample_size = min(500, len(df_noid))
missing_sample = missing_matrix.sample(n=sample_size, random_state=42)

z = missing_sample.T.values
x = missing_sample.index.astype(str)
y = missing_sample.columns.astype(str)

fig2 = go.Figure(
    go.Heatmap(
        z=z,
        x=x,
        y=y,
        colorscale=[[0, "lightblue"], [1, "darkred"]],
        colorbar=dict(title="Missing")
    )
)

fig2.update_layout(
    xaxis_title="Patients",
    yaxis_title="Features",
    margin=dict(l=120, r=40, t=10, b=40)
)

fig2.update_xaxes(showgrid=False)
fig2.update_yaxes(showgrid=False)

fig2.show()


# ======================================================================
# 3. Missing values - Molecular
# ======================================================================

missing_molecular = maf_noid.isnull().sum()
missing_molecular_pct = (missing_molecular / len(maf_noid) * 100).sort_values()

fig3 = go.Figure(
    go.Bar(
        x=missing_molecular_pct.values,
        y=missing_molecular_pct.index,
        orientation="h",
        text=[f"{v:.1f}%" for v in missing_molecular_pct.values],
        textposition="outside"
    )
)

fig3.update_layout(
    xaxis_title="Percentage of Missing Values (%)",
    yaxis_title="Features",
    margin=dict(l=120, r=40, t=10, b=40),
    showlegend=False
)

fig3.update_xaxes(showgrid=False)
fig3.update_yaxes(showgrid=False)

fig3.show()


# ======================================================================
# 4. Mutations per patient
# ======================================================================

mutations_per_patient = maf_df.groupby('ID').size()

fig4 = go.Figure(
    go.Histogram(
        x=mutations_per_patient.values,
        nbinsx=50
    )
)

fig4.update_traces(marker_line_width=1, marker_line_color="black")

fig4.update_layout(
    xaxis_title="Number of Mutations",
    yaxis_title="Number of Patients",
    margin=dict(l=60, r=40, t=10, b=40),
    showlegend=False
)

fig4.update_xaxes(showgrid=False)
fig4.update_yaxes(showgrid=False)

fig4.show()


# ======================================================================
# 5. Clinical vs Molecular comparison (side-by-side)
# ======================================================================

fig5 = make_subplots(rows=1, cols=2)

fig5.add_trace(
    go.Bar(
        x=missing_clinical_pct.values,
        y=missing_clinical_pct.index,
        orientation="h",
        text=[f"{v:.1f}%" for v in missing_clinical_pct.values],
        textposition="outside",
        marker_color="salmon"
    ),
    row=1, col=1
)

fig5.add_trace(
    go.Bar(
        x=missing_molecular_pct.values,
        y=missing_molecular_pct.index,
        orientation="h",
        text=[f"{v:.1f}%" for v in missing_molecular_pct.values],
        textposition="outside",
        marker_color="skyblue"
    ),
    row=1, col=2
)

fig5.update_xaxes(title_text="Missing %", showgrid=False, row=1, col=1)
fig5.update_yaxes(title_text="Features", showgrid=False, row=1, col=1)

fig5.update_xaxes(title_text="Missing %", showgrid=False, row=1, col=2)
fig5.update_yaxes(showgrid=False, row=1, col=2)

fig5.update_layout(
    margin=dict(l=120, r=40, t=10, b=40),
    showlegend=False
)

fig5.show()


# ======================================================================
# 6. Summary stats
# ======================================================================

print("\n=== Summary Statistics ===")

print(f"\nClinical Data (df, excluding ID & CENTER):")
print(f"  Total samples: {len(df_noid)}")
print(f"  Features with missing values: {(missing_clinical > 0).sum()}/{len(missing_clinical)}")
print(f"  Average missing rate: {missing_clinical_pct.mean():.2f}%")

print(f"\nMolecular Data (maf_df, excluding ID):")
print(f"  Total mutations (rows): {len(maf_df)}")
print(f"  Features with missing values: {(missing_molecular > 0).sum()}/{len(missing_molecular)}")
print(f"  Average missing rate: {missing_molecular_pct.mean():.2f}%")



=== Summary Statistics ===

Clinical Data (df, excluding ID & CENTER):
  Total samples: 3323
  Features with missing values: 7/7
  Average missing rate: 7.72%

Molecular Data (maf_df, excluding ID):
  Total mutations (rows): 10935
  Features with missing values: 8/10
  Average missing rate: 0.72%


In [8]:
maf_df2 = pd.read_csv("../../data/molecular_train.csv")
maf_eval2 = pd.read_csv("../../data/molecular_val.csv")

In [9]:
target = ['OS_YEARS', 'OS_STATUS']

# Drop rows where 'OS_YEARS' is NaN if conversion caused any issues
target_df.dropna(subset=target, inplace=True)
target_df['OS_YEARS'] = pd.to_numeric(target_df['OS_YEARS'], errors='coerce')
target_df['OS_STATUS'] = target_df['OS_STATUS'].astype(bool)

# Select features
features = ['ID', 'BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT', 'CYTOGENETICS']

# Create the survival data format
X = df.loc[df['ID'].isin(target_df['ID']), features]
X_eval = df_eval.loc[:, features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_df)


# Impute Missing values

In [10]:
import numpy as np
import pandas as pd
import optuna
from tqdm.notebook import tqdm
import xgboost as xgb

# -------------------------------------------------------------------
# 0. On suppose que X et X_eval existent déjà (DataFrames)
# -------------------------------------------------------------------

# ----------------------
# 1. Colonnes numériques + indicateurs manquants
# ----------------------
num_cols = X.select_dtypes("number").columns

# Indicateurs de NA AVANT imputation
for df_ in (X, X_eval):
    for col in num_cols:
        df_[f"{col}_missing"] = df_[col].isna().astype("int8")

# Matrice numpy (seulement les colonnes numériques originales)
X_num = X[num_cols].to_numpy(dtype=float)
n_rows, n_features = X_num.shape

# ----------------------
# 2. Tuning Optuna PAR FEATURE (1000 trials + barre de progression par feature)
# ----------------------
base_xgb_params = dict(
    objective="reg:squarederror",
    tree_method="hist",
    n_jobs=-1,
    random_state=0,
)

n_trials = 1000       # <---- 1000 TRIALS
mask_frac = 0.1
rng = np.random.RandomState(0)

best_params_per_feature = {}


def make_objective_for_feature(j):
    """Objective Optuna pour la feature j uniquement."""
    def objective(trial):
        # Hyperparamètres XGBoost à optimiser
        xgb_params_trial = {
            "max_depth": trial.suggest_int("max_depth", 2, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        }

        y = X_num[:, j]
        X_features = np.delete(X_num, j, axis=1)

        # on ne garde que les valeurs non-NA de la colonne
        not_nan_y = ~np.isnan(y)
        if not_nan_y.sum() < 20:
            # pas assez de données : on renvoie un score pourri
            return 1e6

        y_obs = y[not_nan_y]
        X_obs = X_features[not_nan_y]

        # masque aléatoire pour l'évaluation
        mask_eval = rng.rand(len(y_obs)) < mask_frac
        if mask_eval.sum() == 0 or (~mask_eval).sum() < 10:
            return 1e6

        X_train = X_obs[~mask_eval]
        y_train = y_obs[~mask_eval]
        X_eval_local = X_obs[mask_eval]
        y_eval_local = y_obs[mask_eval]

        model = xgb.XGBRegressor(
            **base_xgb_params,
            **xgb_params_trial
        )

        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval_local)
        mse = np.mean((y_pred - y_eval_local) ** 2)
        return mse

    return objective


# Boucle Optuna sur chaque feature
for j, col in enumerate(num_cols):
    # Vérification rapide pour éviter de lancer Optuna si trop peu de données
    y_col = X_num[:, j]
    if (~np.isnan(y_col)).sum() < 20:
        continue

    print(f"Tuning feature {j+1}/{len(num_cols)} : {col}")

    study = optuna.create_study(direction="minimize")

    # Barre de progression POUR CETTE FEATURE
    with tqdm(total=n_trials, desc=f"{col}") as pbar:
        def callback(study_, trial_):
            # Affiche la meilleure MSE courante dans la description
            pbar.set_description(f"{col} | best MSE={study_.best_value:.5f}")
            pbar.update(1)

        study.optimize(
            make_objective_for_feature(j),
            n_trials=n_trials,
            callbacks=[callback],
        )

    # On stocke les meilleurs hyperparamètres pour cette feature
    best_params_per_feature[col] = {**base_xgb_params, **study.best_params}

print("Nombre de features tunées :", len(best_params_per_feature))

# ----------------------
# 3. Fit des imputers finaux (un modèle par feature)
# ----------------------
def fit_xgb_imputers_per_feature(X_df, num_cols, best_params_per_feature):
    models = {}
    X_values = X_df[num_cols].to_numpy(dtype=float)

    for j, col in enumerate(num_cols):
        y = X_values[:, j]
        missing_mask = np.isnan(y)
        not_missing = ~missing_mask

        if not_missing.sum() < 20:
            # Pas assez de données pour entraîner un modèle
            continue

        # Si on n'a pas de params Optuna pour cette colonne, on skip
        if col not in best_params_per_feature:
            continue

        X_features = np.delete(X_values, j, axis=1)

        X_train = X_features[not_missing]
        y_train = y[not_missing]

        params = best_params_per_feature[col]
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train)

        models[col] = model

    return models


def transform_with_xgb_imputers(X_df, num_cols, models):
    X_values = X_df[num_cols].to_numpy(dtype=float)

    for j, col in enumerate(num_cols):
        missing_mask = np.isnan(X_values[:, j])

        if missing_mask.any():
            if col in models:
                X_features = np.delete(X_values, j, axis=1)
                X_missing = X_features[missing_mask]
                preds = models[col].predict(X_missing)
                X_values[missing_mask, j] = preds
            else:
                # fallback : médiane de la colonne
                X_values[missing_mask, j] = np.nanmedian(X_values[:, j])

    return pd.DataFrame(X_values, columns=num_cols, index=X_df.index)

# ----------------------
# 4. Imputation finale
# ----------------------
xgb_models = fit_xgb_imputers_per_feature(X, num_cols, best_params_per_feature)

X[num_cols] = transform_with_xgb_imputers(X, num_cols, xgb_models)
X_eval[num_cols] = transform_with_xgb_imputers(X_eval, num_cols, xgb_models)


[I 2025-12-27 13:47:54,965] A new study created in memory with name: no-name-912e5e31-42e1-46a9-842a-09124fc4fee0


Tuning feature 1/6 : BM_BLAST


BM_BLAST:   0%|          | 0/1000 [00:00<?, ?it/s]

[I 2025-12-27 13:47:55,332] Trial 0 finished with value: 52.84779478791412 and parameters: {'max_depth': 10, 'learning_rate': 0.1072021799452273, 'n_estimators': 146, 'subsample': 0.6704689230861887, 'colsample_bytree': 0.5325164744624126}. Best is trial 0 with value: 52.84779478791412.
[I 2025-12-27 13:47:55,739] Trial 1 finished with value: 36.86127656135019 and parameters: {'max_depth': 10, 'learning_rate': 0.039277376164945164, 'n_estimators': 186, 'subsample': 0.8264854048179844, 'colsample_bytree': 0.6933740005769733}. Best is trial 1 with value: 36.86127656135019.
[I 2025-12-27 13:47:55,830] Trial 2 finished with value: 48.9316370750224 and parameters: {'max_depth': 3, 'learning_rate': 0.012791842623646625, 'n_estimators': 234, 'subsample': 0.581681940856739, 'colsample_bytree': 0.9379454939374493}. Best is trial 1 with value: 36.86127656135019.
[I 2025-12-27 13:47:55,924] Trial 3 finished with value: 51.47633566396502 and parameters: {'max_depth': 4, 'learning_rate': 0.05601794

Tuning feature 2/6 : WBC


...

[I 2025-12-27 14:03:55,471] Trial 0 finished with value: 4.249734311974475 and parameters: {'max_depth': 3, 'learning_rate': 0.1247951042354063, 'n_estimators': 242, 'subsample': 0.5826820250997502, 'colsample_bytree': 0.637696964991587}. Best is trial 0 with value: 4.249734311974475.
[I 2025-12-27 14:03:56,342] Trial 1 finished with value: 4.108202653478634 and parameters: {'max_depth': 10, 'learning_rate': 0.017350830082553313, 'n_estimators': 481, 'subsample': 0.5639993368684705, 'colsample_bytree': 0.6906292480763632}. Best is trial 1 with value: 4.108202653478634.
[I 2025-12-27 14:03:56,583] Trial 2 finished with value: 4.811196524993095 and parameters: {'max_depth': 4, 'learning_rate': 0.2669346778150834, 'n_estimators': 403, 'subsample': 0.9942301304036005, 'colsample_bytree': 0.6921497142448823}. Best is trial 1 with value: 4.108202653478634.
[I 2025-12-27 14:03:56,700] Trial 3 finished with value: 3.3003711960870246 and parameters: {'max_depth': 10, 'learning_rate': 0.04542698

Tuning feature 6/6 : PLT


PLT:   0%|          | 0/1000 [00:00<?, ?it/s]

[I 2025-12-27 14:07:47,771] Trial 0 finished with value: 24760.483707132964 and parameters: {'max_depth': 2, 'learning_rate': 0.03133684588987692, 'n_estimators': 79, 'subsample': 0.5348909237532611, 'colsample_bytree': 0.5497490361967765}. Best is trial 0 with value: 24760.483707132964.
[I 2025-12-27 14:07:48,214] Trial 1 finished with value: 28357.968280968696 and parameters: {'max_depth': 7, 'learning_rate': 0.0140541764747719, 'n_estimators': 300, 'subsample': 0.5999757167947637, 'colsample_bytree': 0.585808475115204}. Best is trial 0 with value: 24760.483707132964.
[I 2025-12-27 14:07:48,240] Trial 2 finished with value: 20440.51460400825 and parameters: {'max_depth': 4, 'learning_rate': 0.02702625249916945, 'n_estimators': 52, 'subsample': 0.503018212921209, 'colsample_bytree': 0.9364916337803937}. Best is trial 2 with value: 20440.51460400825.
[I 2025-12-27 14:07:48,589] Trial 3 finished with value: 18009.73053317093 and parameters: {'max_depth': 10, 'learning_rate': 0.019438158

Nombre de features tunées : 6


In [ ]:
X.to_csv('../../data/clinical_train_imputed.csv', index=False)
X_eval.to_csv('../../data/clinical_val_imputed.csv', index=False)

## Enhanced Mutation Features

Computing mutation count (Nmut) along with VAF statistics (average, std, max) and length statistics (average, std, max) for both training and evaluation datasets.

In [ ]:
X = pd.read_csv('../../data/clinical_train_imputed.csv')
X_eval = pd.read_csv('../../data/clinical_val_imputed.csv')

In [120]:
X.isna().sum()

ID                     0
BM_BLAST               0
WBC                    0
ANC                    0
MONOCYTES              0
HB                     0
PLT                    0
CYTOGENETICS         348
BM_BLAST_missing       0
WBC_missing            0
ANC_missing            0
MONOCYTES_missing      0
HB_missing             0
PLT_missing            0
dtype: int64

In [93]:
pd.get_dummies(maf_df, columns=["EFFECT"])
pd.get_dummies(maf_eval, columns=["EFFECT"])

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,VAF,DEPTH,EFFECT_ITD,EFFECT_PTD,EFFECT_frameshift_variant,EFFECT_inframe_codon_gain,EFFECT_inframe_codon_loss,EFFECT_non_synonymous_codon,EFFECT_stop_gained,EFFECT_stop_lost
0,KYW961,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.2620,485.0,False,False,False,False,False,True,False,False
1,KYW142,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.0280,527.0,False,False,False,False,False,True,False,False
2,KYW453,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.2920,277.0,False,False,False,False,False,True,False,False
3,KYW982,1,1747229.0,1747229.0,T,C,GNB1,p.K57E,0.0970,821.0,False,False,False,False,False,True,False,False
4,KYW845,1,36932209.0,36932209.0,G,A,CSF3R,p.Q754X,0.4300,358.0,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,KYW1077,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.4231,NaN,False,True,False,False,False,False,False,False
3085,KYW1084,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.0176,NaN,False,True,False,False,False,False,False,False
3086,KYW1082,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.2273,NaN,False,True,False,False,False,False,False,False
3087,KYW1085,NaN,NaN,NaN,NaN,NaN,MLL,MLL_PTD,0.2941,NaN,False,True,False,False,False,False,False,False


In [94]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler


def compute_mutation_features(maf_df, X_df, top_k_chr=10):
    maf_df = maf_df.copy()

    # --- 0) Vérif de la colonne CHR ---
    if 'CHR' not in maf_df.columns:
        raise ValueError(
            "La colonne 'CHR' est absente de maf_df. "
            "Ne fais pas pd.get_dummies(maf_df, columns=['CHR']) "
            "avant d'appeler compute_mutation_features, ou passe la version brute."
        )

    # --- 1) Longueur de la mutation & 'deletion length' ---
    maf_df['LEN'] = maf_df['END'] - maf_df['START'] + 1
    maf_df['DELLEN'] = maf_df['LEN'] - maf_df['REF'].apply(lambda x: len(str(x)))

    # --- 1bis) CHR : ne garder que les top_k_chr pour les features de comptage ---
    # (mais CHR_nunique sera quand même calculé sur tous les CHR)
    top_chr = maf_df['CHR'].value_counts().nlargest(top_k_chr).index
    unique_chr = sorted(top_chr)

    # Colonnes one-hot d'EFFECT déjà présentes
    effect_dummy_cols = [c for c in maf_df.columns if c.startswith('EFFECT_')]

    # --- 2) Dictionnaire d'agrégation de base ---
    agg_dict = {
        'Nmut': ('ID', 'size'),
        'VAF_avg': ('VAF', 'mean'),
        'VAF_std': ('VAF', 'std'),
        'VAF_max': ('VAF', 'max'),
        'LEN_avg': ('LEN', 'mean'),
        'LEN_max': ('LEN', 'max'),
        'DELLEN_sum': ('DELLEN', 'sum'),
        'DEPTH_avg': ('DEPTH', 'mean'),
        'DEPTH_std': ('DEPTH', 'std'),
        'DEPTH_max': ('DEPTH', 'max'),
        'DEPTH_min': ('DEPTH', 'min'),
        'CHR_nunique': ('CHR', 'nunique'),
        'EFFECT_nunique': ('EFFECT', 'nunique'),
        'EFFECT_FV_count': ('EFFECT', lambda x: (x == 'frameshift_variant').sum()),
        'EFFECT_SG_count': ('EFFECT', lambda x: (x == 'stop_gained').sum()),
        'EFFECT_NS_count': ('EFFECT', lambda x: (x == 'non_synonymous_codon').sum()),
    }

    # --- 3) Colonnes CHR_*_count seulement pour les top_k_chr ---
    for ch in unique_chr:
        col_name = f'CHR_{ch}_count'
        agg_dict[col_name] = ('CHR', lambda x, val=ch: (x == val).sum())

    # --- 4) Agrégation des dummies EFFECT_* au niveau patient ---
    # -> EFFECT_*_count = nombre de mutations de ce type par patient
    for col in effect_dummy_cols:
        new_name = f'{col}_count'
        agg_dict[new_name] = (col, 'sum')

    # --- 5) Agrégation globale par ID ---
    tmp = maf_df.groupby('ID').agg(**agg_dict).reset_index()

    # Remplir NaN des std (quand une seule mutation par patient)
    for std_col in ['VAF_std', 'DEPTH_std']:
        if std_col in tmp.columns:
            tmp[std_col] = tmp[std_col].fillna(0)

    # --- 6) Loss-of-function : EFFECT_LOF_count & EFFECT_LOF_ratio ---
    # On repère les colonnes *_count associées à frameshift / stop_gained
    lof_effect_cols = [
        c for c in tmp.columns
        if c.startswith('EFFECT_')
        and c.endswith('_count')
        and (
            'frameshift_variant' in c
            or 'stop_gained' in c
            # tu peux ajouter d'autres patterns ici si besoin : 'splice', etc.
        )
    ]

    if len(lof_effect_cols) > 0:
        tmp['EFFECT_LOF_count'] = tmp[lof_effect_cols].sum(axis=1)
    else:
        tmp['EFFECT_LOF_count'] = 0

    tmp['EFFECT_LOF_ratio'] = np.where(
        tmp['Nmut'] > 0,
        tmp['EFFECT_LOF_count'] / tmp['Nmut'],
        0.0
    )

    # --- 7) Merge avec X_df ---
    X_w_mutation = X_df.merge(tmp, on='ID', how='left')

    # Colonnes ajoutées par l'agg (à remplir avec 0 pour les patients sans mutation)
    new_cols = [c for c in tmp.columns if c != 'ID']
    X_w_mutation[new_cols] = X_w_mutation[new_cols].fillna(0)

    return X_w_mutation


# ============================================================
# ================ UTILISATION TRAIN / EVAL ==================
# ============================================================

# On suppose que maf_df, maf_eval, X, X_eval existent déjà.

# 1) Ajout des features mutationnelles au train et à l'eval
X_w_mutation = compute_mutation_features(maf_df, X)
X_eval_w_mutation = compute_mutation_features(maf_eval, X_eval)

# 2) Construction de la liste des features de mutation (à partir du train)
base_mutation_features = [
    'Nmut', 'VAF_avg', 'VAF_std', 'VAF_max',
    'LEN_avg', 'LEN_max', 'DELLEN_sum',
    'DEPTH_avg', 'DEPTH_std', 'DEPTH_max', 'DEPTH_min',
    'CHR_nunique',
    'EFFECT_nunique', 'EFFECT_FV_count', 'EFFECT_SG_count', 'EFFECT_NS_count',
    'EFFECT_LOF_count', 'EFFECT_LOF_ratio',  # nouveau
]

# Colonnes CHR_*_count dans le TRAIN
chr_count_cols = [
    c for c in X_w_mutation.columns
    if c.startswith('CHR_') and c.endswith('_count')
]

# Colonnes EFFECT_*_count générées automatiquement depuis les dummies
effect_count_cols = [
    c for c in X_w_mutation.columns
    if c.startswith('EFFECT_') and c.endswith('_count')
    and c not in ['EFFECT_FV_count', 'EFFECT_SG_count', 'EFFECT_NS_count', 'EFFECT_LOF_count']
]

mutation_features = base_mutation_features + chr_count_cols + effect_count_cols

# 3) Harmoniser les colonnes entre train et eval
for col in mutation_features:
    if col not in X_w_mutation.columns:
        X_w_mutation[col] = 0
    if col not in X_eval_w_mutation.columns:
        X_eval_w_mutation[col] = 0

# (Optionnel) s'assurer que les colonnes sont bien dans le même ordre
X_w_mutation = X_w_mutation.copy()
X_eval_w_mutation = X_eval_w_mutation.copy()
X_w_mutation[mutation_features] = X_w_mutation[mutation_features]
X_eval_w_mutation[mutation_features] = X_eval_w_mutation[mutation_features]

# 4) RobustScaler sur ces features
mutation_scaler = RobustScaler()
print(
    f"Fitting RobustScaler for mutation features ({len(mutation_features)} features) "
    "on training data, transforming training and evaluation data."
)
X_w_mutation[mutation_features] = mutation_scaler.fit_transform(X_w_mutation[mutation_features])
X_eval_w_mutation[mutation_features] = mutation_scaler.transform(X_eval_w_mutation[mutation_features])


Fitting RobustScaler for mutation features (28 features) on training data, transforming training and evaluation data.


# Processing Cytogenetics

sorte de "rules engine" basé sur la littérature clinique AML pour extraire des features pronostiques clés depuis l’ISCN. On détecte

* **Monosomal karyotype (MK)**: au moins 2 monosomies autosomiques, ou 1 monosomie autosomique + au moins une anomalie structurale. C’est franchement mauvais pronostic. ([PubMed Central][1])
* **Complex karyotype**: typiquement ≥ 3 anomalies cytogénétiques indépendantes, pronostic défavorable. ([Cancer Network][2])
* **Chromosomes 5 et 7**: perte de 5 ou 7, ou del(5q)/del(7q), classés défavorables dans ELN 2022. ([ASH Publications][3])
* **+8 (trisomie 8)**: très fréquente, plutôt risque intermédiaire en AML. ([MDPI][4])
* **Translocations favorables classiques**: t(8;21), inv(16)/t(16;16); APL t(15;17) est à part. On les isole comme features car elles portent un signal fort. ([ASH Publications][3])
* **Proportions clonales** via les crochets `[n]` pour calculer la part de métaphases portant une anomalie donnée. La sémantique des notations vient de l’ISCN. ([PubMed][5])

Pour chaque karyotype, on construit:

* `has_any_abnormality`, `n_events`, `n_chromosomes_altered`
* `has_minus5_or_del5q`, `has_minus7_or_del7q`, `has_plus8`
* `has_t_8_21`, `has_inv16_or_t_16_16`, `has_t_15_17`
* `is_complex_karyotype`, `is_monosomal_karyotype`
* `total_metaphases` et des **proportions clonales**: `prop_any_abnormal`, `prop_adverse_5_7`, `prop_plus8`, etc.
* Option: `eln_like_flag_adverse_cyto` basé ici sur 5/7 ou MK ou complex.

[1]: https://pmc.ncbi.nlm.nih.gov/articles/PMC3069222/?utm_source=chatgpt.com "Acute myeloid leukemia with monosomal karyotype at the ..."
[2]: https://www.cancernetwork.com/view/unfavorable-complex-and-monosomal-karyotypes-most-challenging-forms-acute-myeloid-leukemia?utm_source=chatgpt.com "Unfavorable, Complex, and Monosomal Karyotypes"
[3]: https://ashpublications.org/blood/article/140/12/1345/485817/Diagnosis-and-management-of-AML-in-adults-2022?utm_source=chatgpt.com "Diagnosis and management of AML in adults - ASH Publications"
[4]: https://www.mdpi.com/2072-6694/13/22/5679?utm_source=chatgpt.com "Risk Stratification, Measurable Residual Disease, and ..."
[5]: https://pubmed.ncbi.nlm.nih.gov/34839499/?utm_source=chatgpt.com "[Introduction and interpretation of the updated contents of ..."


In [95]:
import re
from typing import List, Dict
import pandas as pd

# --- REGEX patterns ---
_ISCN_EVENT_RE = re.compile(r'(del|dup|inv|ins|i|t|add|der)\s*\(', re.IGNORECASE)
_MONOSOMY_RE   = re.compile(r'(?<![pq])-(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)
_TRISOMY_RE    = re.compile(r'(?<![pq])\+(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)
_CHR_NUM_RE    = re.compile(r'(?<![pq])(\d{1,2}|X|Y)(?![pq])', re.IGNORECASE)

# specific adverse/favorable (de base)
_MINUS5_OR_DEL5Q_RE = re.compile(r'-(?:5)(?![pq])|del\s*\(\s*5\s*\)\s*\(\s*q', re.IGNORECASE)
_MINUS7_OR_DEL7Q_RE = re.compile(r'-(?:7)(?![pq])|del\s*\(\s*7\s*\)\s*\(\s*q', re.IGNORECASE)
_PLUS8_RE           = re.compile(r'\+8(?![pq])', re.IGNORECASE)
_T_8_21_RE          = re.compile(r't\s*\(\s*8\s*;\s*21\s*\)', re.IGNORECASE)
_INV16_OR_T_16_16_RE= re.compile(r'(inv\s*\(\s*16\s*\)|t\s*\(\s*16\s*;\s*16\s*\))', re.IGNORECASE)
_T_15_17_RE         = re.compile(r't\s*\(\s*15\s*;\s*17\s*\)', re.IGNORECASE)
_STRUCTURAL_RE      = re.compile(r'(del|dup|inv|ins|i|t|add|der)\s*\(', re.IGNORECASE)

# nouveaux patterns ELN-like
_INV3_OR_T3_3_RE    = re.compile(r'(inv\s*\(\s*3\s*\)\s*\(q21q26\)|t\s*\(\s*3\s*;\s*3\s*\)\s*\(q21;q26\))', re.IGNORECASE)
_T_6_9_RE           = re.compile(r't\s*\(\s*6\s*;\s*9\s*\)\s*\(p23;q34\)', re.IGNORECASE)
_T_9_22_RE          = re.compile(r't\s*\(\s*9\s*;\s*22\s*\)\s*\(q34;q11\)', re.IGNORECASE)
_ABN_17P_RE         = re.compile(r'(del\s*\(\s*17\s*\)\s*\(\s*p|del\s*\(\s*17p\s*\)|-17(?![pq])|add\s*\(\s*17\s*\)\s*\(\s*p)', re.IGNORECASE)

# baseline chr count (ex: 46,XX,del(5q)... -> 46)
_BASELINE_CHR_RE    = re.compile(r'^\s*(\d{2})\s*,', re.IGNORECASE)

# normal karyotype (simple)
_NORMAL_KARYO_RE    = re.compile(r'^\s*46\s*,\s*(XX|XY)\s*(\[\d+\])?\s*$', re.IGNORECASE)


# --- helpers ---
def _split_clones(karyo: str) -> List[str]:
    """Split ISCN string into clones separated by '/'."""
    return [c.strip() for c in str(karyo).split('/') if c.strip()]


def _extract_metaphases(clone: str) -> int:
    """Extract number of metaphases from [n] in clone."""
    m = re.search(r'\[(\d+)\]', clone)
    return int(m.group(1)) if m else 0


def _count_events(clone: str) -> int:
    """
    Count events in a clone:
    structural + trisomies + autosomal monosomies (ignore -Y).
    """
    n_struct = len(_ISCN_EVENT_RE.findall(clone))
    n_mono   = len(_MONOSOMY_RE.findall(clone))
    n_tri    = len(_TRISOMY_RE.findall(clone))
    n_mono_minusY = len(re.findall(r'(?<![pq])-(?:Y)(?![pq])', clone, flags=re.IGNORECASE))
    return n_struct + n_tri + max(n_mono - n_mono_minusY, 0)


def _chromosomes_altered(clone: str) -> int:
    """Number of distinct autosomes/sex chromosomes altered (ignore Y)."""
    nums = set()
    for m in _MONOSOMY_RE.finditer(clone):
        nums.add(m.group(1).upper())
    for m in _TRISOMY_RE.finditer(clone):
        nums.add(m.group(1).upper())
    for ev in re.finditer(r'(del|dup|inv|ins|i|t|add|der)\s*\(([^)]+)\)', clone, flags=re.IGNORECASE):
        for x in re.split(r'[;,\s]+', ev.group(2)):
            if _CHR_NUM_RE.fullmatch(x.strip()):
                nums.add(x.strip().upper())
    nums.discard('Y')
    return len(nums)


def _has_structural(clone: str) -> bool:
    return bool(_STRUCTURAL_RE.search(clone))


def _autosomic_monosomies(clone: str) -> List[int]:
    """List of autosomal monosomies in clone (ignore X/Y)."""
    return [int(m.group(1)) for m in _MONOSOMY_RE.finditer(clone) if m.group(1).upper() not in ('X', 'Y')]

def _autosomic_trisomies(clone: str) -> List[int]:
    """Liste des trisomies autosomiques dans le clone (ignore X/Y)."""
    return [
        int(m.group(1))
        for m in _TRISOMY_RE.finditer(clone)
        if m.group(1).upper() not in ('X', 'Y')
    ]

def _is_monosomal_karyotype(karyo: str) -> bool:
    for c in _split_clones(karyo):
        autos = _autosomic_monosomies(c)
        has_struct = _has_structural(c)
        if len(autos) >= 2:
            return True
        if len(autos) >= 1 and has_struct:
            return True
    return False



def _is_complex_karyotype(karyo: str) -> bool:
    for c in _split_clones(karyo):
        c_wo_minusY = re.sub(r'(?<![pq])-(?:Y)(?![pq])', '', c, flags=re.IGNORECASE)
        if _count_events(c_wo_minusY) >= 3:
            return True
    return False


def _extract_baseline_chr_count(karyo: str) -> int:
    """
    Extract baseline chromosome number at start of ISCN (ex: 46,XX,... -> 46).
    Return -1 if not found.
    """
    if not isinstance(karyo, str):
        return -1
    m = _BASELINE_CHR_RE.match(karyo)
    if not m:
        return -1
    try:
        return int(m.group(1))
    except ValueError:
        return -1


def _clone_flags(clone: str) -> Dict[str, bool]:
    """
    Flags at clone-level (adverse/favorable, counts, etc.)
    """
    return {
        # anomalies "classiques"
        'minus5_or_del5q': bool(_MINUS5_OR_DEL5Q_RE.search(clone)),
        'minus7_or_del7q': bool(_MINUS7_OR_DEL7Q_RE.search(clone)),
        'plus8':           bool(_PLUS8_RE.search(clone)),
        't_8_21':          bool(_T_8_21_RE.search(clone)),
        'inv16_or_t_16_16':bool(_INV16_OR_T_16_16_RE.search(clone)),
        't_15_17':         bool(_T_15_17_RE.search(clone)),

        # anomalies ELN "rares" supplémentaires
        'inv3_or_t3_3':    bool(_INV3_OR_T3_3_RE.search(clone)),
        't_6_9':           bool(_T_6_9_RE.search(clone)),
        't_9_22':          bool(_T_9_22_RE.search(clone)),
        'abn17p':          bool(_ABN_17P_RE.search(clone)),

        # structure & events
        'has_structural':        _has_structural(clone),
        'events_count':          _count_events(clone),
        'chrs_altered':          _chromosomes_altered(clone),
        'has_any_abn':           bool(_ISCN_EVENT_RE.search(clone) or
                                      _MONOSOMY_RE.search(clone) or
                                      _TRISOMY_RE.search(clone)),
        # counts plus détaillés
        'n_monosomies':          len(_MONOSOMY_RE.findall(clone)),
        'n_trisomies':           len(_TRISOMY_RE.findall(clone)),
        'n_structural_events':   len(_ISCN_EVENT_RE.findall(clone)),
    }


# --- main featurizer ---
def add_cytogenetics_features(df: pd.DataFrame, col: str = "CYTOGENETICS") -> pd.DataFrame:
    rows = []

    for k in df[col]:
        # --- cas "missing / not done" ---
        if not isinstance(k, str) or not k.strip() or k.strip().lower() in {"nan", "na", "nd", "notdone", "failed", "failure"}:
            rows.append({
                'is_cyto_missing_or_failed': 1,
                'is_normal_karyotype': 0,
                'is_abnormal_karyotype': 0,
                'has_any_abnormality': 0,
                'n_events': 0,
                'n_chromosomes_altered': 0,
                'n_monosomies_total': 0,
                'n_trisomies_total': 0,
                'n_structural_events_total': 0,
                'has_minus5_or_del5q': 0,
                'has_minus7_or_del7q': 0,
                'has_plus8': 0,
                'has_t_8_21': 0,
                'has_inv16_or_t_16_16': 0,
                'has_t_15_17': 0,
                'has_inv3_or_t3_3': 0,
                'has_t_6_9': 0,
                'has_t_9_22': 0,
                'has_abn17p': 0,
                'is_monosomal_karyotype': 0,
                'is_complex_karyotype': 0,
                'eln_like_flag_adverse_cyto': 0,
                'eln_like_flag_favorable_cyto': 0,
                'eln_like_flag_intermediate_cyto': 0,
                'eln_like_risk_cyto': -1,   # -1 = missing
                'baseline_chr_count': -1,
                'is_hypodiploid': 0,
                'is_hyperdiploid': 0,
                'is_near_tetraploid': 0,
                'total_metaphases': 0,
                'max_clone_size': 0.0,
                'max_adverse_clone_size': 0.0,
                'has_small_adverse_subclone': 0,
                'prop_any_abnormal': 0.0,
                'prop_adverse_5_7': 0.0,
                'prop_plus8': 0.0,
                'prop_favorable_core': 0.0,

                # --- NEW ---
                'n_autosomal_monosomies': 0,
                'n_autosomal_trisomies': 0,
                'worst_clone_events': 0,
                'worst_clone_is_adverse': 0,
                'has_any_rare_adverse_cyto': 0,
            })
            continue


        clones = _split_clones(k)
        clone_info = []
        total_meta_known = 0

        n_autosomal_mono_tot = 0  
        n_autosomal_tri_tot = 0  


        for c in clones:
            n_meta = _extract_metaphases(c)
            flags = _clone_flags(c)

            # NEW: compter monosomies / trisomies autosomiques
            n_autosomal_mono_tot += len(_autosomic_monosomies(c))
            n_autosomal_tri_tot += len(_autosomic_trisomies(c))

            clone_info.append((c, n_meta, flags))
            total_meta_known += n_meta



        # base stats
        any_abn   = any(f['has_any_abn'] for _, _, f in clone_info)
        n_events  = sum(f['events_count'] for _, _, f in clone_info)
        n_chrs    = max([f['chrs_altered'] for _, _, f in clone_info] + [0])

        n_mono_tot   = sum(f['n_monosomies'] for _, _, f in clone_info)
        n_tris_tot   = sum(f['n_trisomies'] for _, _, f in clone_info)
        n_struct_tot = sum(f['n_structural_events'] for _, _, f in clone_info)

        # anomalies spécifiques
        has_minus5_or_del5q = any(f['minus5_or_del5q'] for _, _, f in clone_info)
        has_minus7_or_del7q = any(f['minus7_or_del7q'] for _, _, f in clone_info)
        has_plus8           = any(f['plus8'] for _, _, f in clone_info)
        has_t_8_21          = any(f['t_8_21'] for _, _, f in clone_info)
        has_inv16_or_t_16_16= any(f['inv16_or_t_16_16'] for _, _, f in clone_info)
        has_t_15_17         = any(f['t_15_17'] for _, _, f in clone_info)
        has_inv3_or_t3_3    = any(f['inv3_or_t3_3'] for _, _, f in clone_info)
        has_t_6_9           = any(f['t_6_9'] for _, _, f in clone_info)
        has_t_9_22          = any(f['t_9_22'] for _, _, f in clone_info)
        has_abn17p          = any(f['abn17p'] for _, _, f in clone_info)

        # MK / complex
        is_mk  = _is_monosomal_karyotype(k)
        is_ck  = _is_complex_karyotype(k)

        # baseline chr + ploidie
        baseline_chr = _extract_baseline_chr_count(k)
        is_hypo      = int(baseline_chr != -1 and baseline_chr < 46)
        is_hyper     = int(baseline_chr != -1 and 46 < baseline_chr < 50)
        is_near_tet  = int(baseline_chr != -1 and baseline_chr >= 80)

        # normal / abnormal / missing
        is_normal    = int(bool(_NORMAL_KARYO_RE.match(k)))
        is_abnormal  = int(not is_normal and any_abn)
        is_missing   = 0  # déjà filtré avant

        # ELN-like : favorable / adverse / intermédiaire
        eln_favorable = bool(has_t_8_21 or has_inv16_or_t_16_16 or has_t_15_17)
        eln_adverse_basic = bool(is_mk or is_ck or has_minus5_or_del5q or has_minus7_or_del7q)
        eln_adverse_extended = bool(
            eln_adverse_basic
            or has_inv3_or_t3_3
            or has_t_6_9
            or has_t_9_22
            or has_abn17p
        )

        if is_missing:
            eln_risk = -1
        else:
            if eln_adverse_extended:
                eln_risk = 2
            elif eln_favorable:
                eln_risk = 0
            else:
                # le reste (avec info cytogénétique) = intermédiaire
                eln_risk = 1
        has_any_rare_adverse_cyto = int(
            has_inv3_or_t3_3 or has_t_6_9 or has_t_9_22 or has_abn17p
        )

        # 2) worst clone events + is_adverse (parmi les clones, celui avec max events_count)
        worst_clone_events = 0
        worst_clone_is_adverse = 0
        if clone_info:
            max_events = -1
            worst_is_adverse = 0
            for _, _, f in clone_info:
                ev = f['events_count']
                # logique "extended adverse" clone-level, comme tu l'utilises déjà
                is_extended_adverse_clone = (
                    f['minus5_or_del5q'] or f['minus7_or_del7q'] or
                    f['inv3_or_t3_3'] or f['t_6_9'] or
                    f['t_9_22'] or f['abn17p']
                )
                if ev > max_events:
                    max_events = ev
                    worst_is_adverse = int(is_extended_adverse_clone)
            worst_clone_events = int(max_events if max_events >= 0 else 0)
            worst_clone_is_adverse = int(worst_is_adverse)

        # clonality helpers
        def _prop(cond_fn):
            if total_meta_known == 0:
                return 0.0
            pos = sum(n_meta for _, n_meta, f in clone_info if n_meta and cond_fn(f))
            return pos / total_meta_known if total_meta_known else 0.0

        # proportion de clones anormaux, défavorables, etc.
        prop_any_abnormal = float(_prop(lambda f: f['has_any_abn']))
        prop_adverse_5_7  = float(_prop(lambda f: f['minus5_or_del5q'] or f['minus7_or_del7q']))
        prop_plus8        = float(_prop(lambda f: f['plus8']))
        prop_favorable_core = float(_prop(lambda f: f['t_8_21'] or f['inv16_or_t_16_16']))

        # clonality plus fine : max clone, max clone "adverse"
        max_clone_prop = 0.0
        max_adverse_prop = 0.0
        has_small_adverse_subclone = 0

        if total_meta_known > 0:
            for _, n_meta, f in clone_info:
                if not n_meta:
                    continue
                p = n_meta / total_meta_known
                if p > max_clone_prop:
                    max_clone_prop = p

                is_extended_adverse_clone = (
                    f['minus5_or_del5q'] or f['minus7_or_del7q'] or
                    f['inv3_or_t3_3'] or f['t_6_9'] or f['t_9_22'] or f['abn17p']
                )
                if is_extended_adverse_clone:
                    if p > max_adverse_prop:
                        max_adverse_prop = p
                    if 0.0 < p < 0.3:
                        has_small_adverse_subclone = 1

        rows.append({
            'is_cyto_missing_or_failed': int(is_missing),
            'is_normal_karyotype': int(is_normal),
            'is_abnormal_karyotype': int(is_abnormal),
            'has_any_abnormality': int(any_abn),
            'n_events': int(n_events),
            'n_chromosomes_altered': int(n_chrs),
            'n_monosomies_total': int(n_mono_tot),
            'n_trisomies_total': int(n_tris_tot),
            'n_structural_events_total': int(n_struct_tot),
            'has_minus5_or_del5q': int(has_minus5_or_del5q),
            'has_minus7_or_del7q': int(has_minus7_or_del7q),
            'has_plus8': int(has_plus8),
            'has_t_8_21': int(has_t_8_21),
            'has_inv16_or_t_16_16': int(has_inv16_or_t_16_16),
            'has_t_15_17': int(has_t_15_17),
            'has_inv3_or_t3_3': int(has_inv3_or_t3_3),
            'has_t_6_9': int(has_t_6_9),
            'has_t_9_22': int(has_t_9_22),
            'has_abn17p': int(has_abn17p),
            'is_monosomal_karyotype': int(is_mk),
            'is_complex_karyotype': int(is_ck),
            'eln_like_flag_adverse_cyto': int(eln_adverse_extended),
            'eln_like_flag_favorable_cyto': int(eln_favorable),
            'eln_like_flag_intermediate_cyto': int(eln_risk == 1),
            'eln_like_risk_cyto': int(eln_risk),
            'baseline_chr_count': int(baseline_chr),
            'is_hypodiploid': int(is_hypo),
            'is_hyperdiploid': int(is_hyper),
            'is_near_tetraploid': int(is_near_tet),
            'total_metaphases': int(total_meta_known),
            'max_clone_size': float(max_clone_prop),
            'max_adverse_clone_size': float(max_adverse_prop),
            'has_small_adverse_subclone': int(has_small_adverse_subclone),
            'prop_any_abnormal': float(prop_any_abnormal),
            'prop_adverse_5_7': float(prop_adverse_5_7),
            'prop_plus8': float(prop_plus8),
            'prop_favorable_core': float(prop_favorable_core),

            # --- NEW ---
            'n_autosomal_monosomies': int(n_autosomal_mono_tot),
            'n_autosomal_trisomies': int(n_autosomal_tri_tot),
            'worst_clone_events': int(worst_clone_events),
            'worst_clone_is_adverse': int(worst_clone_is_adverse),
            'has_any_rare_adverse_cyto': int(has_any_rare_adverse_cyto),
        })

    features_df = pd.DataFrame(rows, index=df.index)
    # On enlève la colonne brute CYTOGENETICS (comme tu le faisais)
    return pd.concat([df.copy(), features_df], axis=1).drop(columns=[col])


In [96]:
# === Enrichissement avec les features cytogénétiques ===
X_enhanced = add_cytogenetics_features(X_w_mutation)
X_eval_enhanced = add_cytogenetics_features(X_eval_w_mutation)

# Liste élargie de features cytogénétiques (toutes créées par add_cytogenetics_features)
cytogenetics_features = [
    # statut global / qualité
    'is_cyto_missing_or_failed',
    'is_normal_karyotype',
    'is_abnormal_karyotype',

    # complexité / volume d’anomalies
    'has_any_abnormality',
    'n_events',
    'n_chromosomes_altered',
    'n_monosomies_total',
    'n_trisomies_total',
    'n_structural_events_total',

    # anomalies spécifiques défavorables / favorables
    'has_minus5_or_del5q',
    'has_minus7_or_del7q',
    'has_plus8',
    'has_t_8_21',
    'has_inv16_or_t_16_16',
    'has_t_15_17',
    'has_inv3_or_t3_3',
    'has_t_6_9',
    'has_t_9_22',
    'has_abn17p',

    # MK / complexe
    'is_monosomal_karyotype',
    'is_complex_karyotype',

    # résumé type ELN-like
    'eln_like_flag_adverse_cyto',
    'eln_like_flag_favorable_cyto',
    'eln_like_flag_intermediate_cyto',
    'eln_like_risk_cyto',

    # ploidie
    'baseline_chr_count',
    'is_hypodiploid',
    'is_hyperdiploid',
    'is_near_tetraploid',

    # clonalité
    'total_metaphases',
    'max_clone_size',
    'max_adverse_clone_size',
    'has_small_adverse_subclone',

    # proportions clonales sur certains patterns
    'prop_any_abnormal',
    'prop_adverse_5_7',
    'prop_plus8',
    'prop_favorable_core',

    # --- NEW ---
    'n_autosomal_monosomies',
    'n_autosomal_trisomies',
    'worst_clone_events',
    'worst_clone_is_adverse',
    'has_any_rare_adverse_cyto',
]

# === 1) Détection des features quasi constantes (>=95% identique) sur le TRAIN ===
nearly_constant_features = []
for col in cytogenetics_features:
    value_counts = X_enhanced[col].value_counts(dropna=False)
    if len(value_counts) > 0:
        max_proportion = value_counts.iloc[0] / len(X_enhanced)
        protected = {
    'has_minus5_or_del5q', 'has_minus7_or_del7q', 'has_abn17p',
    'has_inv3_or_t3_3', 'has_t_6_9', 'has_t_9_22',
    'is_monosomal_karyotype', 'is_complex_karyotype',
    'has_t_15_17', 'has_t_8_21', 'has_inv16_or_t_16_16'
}

    if max_proportion >= 0.95 and col not in protected:
        nearly_constant_features.append(col)
        print(
            f"Feature '{col}' is nearly constant "
            f"({max_proportion:.2%} of samples have the same value) "
            "-> will be removed."
        )


# === 2) Suppression des features quasi constantes dans train et eval ===
X_enhanced = X_enhanced.drop(columns=nearly_constant_features)
X_eval_enhanced = X_eval_enhanced.drop(columns=nearly_constant_features)

# Mettre à jour la liste des features cyto réellement utilisées
cytogenetics_features = [f for f in cytogenetics_features if f not in nearly_constant_features]

# === 3) RobustScaler sur les features cyto restantes ===
cytogenetics_scaler = RobustScaler()
print(
    f"Fitting RobustScaler for {len(cytogenetics_features)} cytogenetics features "
    "on training data, transforming training and evaluation data."
)
X_enhanced[cytogenetics_features] = cytogenetics_scaler.fit_transform(
    X_enhanced[cytogenetics_features]
)
X_eval_enhanced[cytogenetics_features] = cytogenetics_scaler.transform(
    X_eval_enhanced[cytogenetics_features]
)


Feature 'eln_like_flag_favorable_cyto' is nearly constant (99.97% of samples have the same value) -> will be removed.
Feature 'is_near_tetraploid' is nearly constant (99.94% of samples have the same value) -> will be removed.
Feature 'has_small_adverse_subclone' is nearly constant (96.56% of samples have the same value) -> will be removed.
Feature 'prop_favorable_core' is nearly constant (100.00% of samples have the same value) -> will be removed.
Feature 'has_any_rare_adverse_cyto' is nearly constant (97.64% of samples have the same value) -> will be removed.
Fitting RobustScaler for 37 cytogenetics features on training data, transforming training and evaluation data.


In [97]:
df_enhanced = X_enhanced.merge(target_df, on='ID', how='left')
df_eval_enhanced = X_eval_enhanced


In [98]:
import pandas as pd

import pandas as pd

def add_gene_features(df_clinical_enhanced, df_molecular, gene_list=None, top_k=150):
    """
    Ajoute des features one-hot pour les gènes :
    - Si gene_list est None : utilise les top_k gènes les plus fréquents dans df_molecular
      (fréquence = nombre de patients distincts dans lesquels le gène apparaît).
    - Sinon : force l'usage de gene_list (pour que train et val aient les mêmes colonnes)
    """
    # 1) Définir la liste de gènes de référence
    if gene_list is None:
        # fréquence par gène = nb d'ID distincts où le gène est présent
        gene_counts = (
            df_molecular[['ID', 'GENE']]
            .drop_duplicates()['GENE']
            .value_counts()
        )
        # on garde les top_k gènes les plus fréquents
        gene_list = gene_counts.nlargest(top_k).index.tolist()
        print(f"Nombre de gènes utilisés (top {top_k}) : {len(gene_list)}")

    # 2) Ne garder que les lignes correspondant aux gènes de la liste
    df_filtered = df_molecular[df_molecular['GENE'].isin(gene_list)].copy()

    # 3) Table ID x GENE (one-hot)
    gene_pivot = pd.crosstab(df_filtered['ID'], df_filtered['GENE'])

    # 4) S'assurer que toutes les colonnes de gene_list existent
    for g in gene_list:
        if g not in gene_pivot.columns:
            gene_pivot[g] = 0

    # 5) Réordonner les colonnes exactement selon gene_list
    gene_pivot = gene_pivot[gene_list]

    # 6) Binariser et renommer les colonnes
    gene_pivot = (gene_pivot > 0).astype(int)
    gene_pivot.columns = [f'Gene_{col}' for col in gene_pivot.columns]

    # 7) Merge avec les features cliniques
    df_final = df_clinical_enhanced.merge(gene_pivot, on='ID', how='left')

    # 8) Remplir les NaN des colonnes de gènes avec 0
    new_cols = [c for c in df_final.columns if c.startswith('Gene_')]
    df_final[new_cols] = df_final[new_cols].fillna(0)

    return df_final, gene_list



# ======================================================================
# Lecture des données
# ======================================================================
mol_train_raw = pd.read_csv("../../data/molecular_train.csv")
mol_val_raw   = pd.read_csv("../../data/molecular_val.csv")

# df_enhanced et df_df_val_pivot sont tes jeux cliniques déjà préparés
# df_enhanced      : train clinique
# df_df_val_pivotal clinique

# ======================================================================
# Construction des features gènes pour train et val
# ======================================================================
df_train_pivot, gene_list_ref = add_gene_features(
    df_clinical_enhanced=df_enhanced,
    df_molecular=mol_train_raw,
    gene_list=None,  # on déduit les top 150 gènes du train
    top_k=150
)

df_val_pivot, _ = add_gene_features(
    df_clinical_enhanced=df_eval_enhanced,
    df_molecular=mol_val_raw,
    gene_list=gene_list_ref  # même liste (top 70) que le train
)


# ======================================================================
# Harmonisation des colonnes entre train et val
# ======================================================================

# Colonnes de gènes (identiques pour les deux par construction)
gene_cols = [c for c in df_train_pivot.columns if c.startswith('Gene_')]

# Colonnes cliniques pour les données d'entraînement (incluant OS_YEARS et OS_STATUS)
train_clinical_cols = [c for c in df_train_pivot.columns if not c.startswith('Gene_')]

# Colonnes cliniques pour les données de validation (excluant OS_YEARS et OS_STATUS)
val_clinical_cols = [c for c in df_val_pivot.columns if not c.startswith('Gene_')]

# Pour l'harmonisation, on exclut temporairement OS_YEARS et OS_STATUS
train_clinical_cols_for_common = [c for c in train_clinical_cols if c not in ['OS_YEARS', 'OS_STATUS']]

# On garde uniquement les colonnes cliniques communes (sans les targets)
common_clinical = sorted(set(train_clinical_cols_for_common).intersection(val_clinical_cols))

# Ordre final pour les données d'entraînement : colonnes cliniques communes + gènes + targets
final_features_train = common_clinical + gene_cols + ['OS_YEARS', 'OS_STATUS']

# Ordre final pour les données de validation : colonnes cliniques communes + gènes (pas de targets)
final_features_val = common_clinical + gene_cols

# Sélection des colonnes dans le bon ordre pour train et val
df_train_pivot = df_train_pivot[final_features_train]
df_val_pivot = df_val_pivot[final_features_val]

print(f"Nombre de colonnes finales train (avec targets) : {len(final_features_train)}")
print(f"Nombre de colonnes finales val (sans targets) : {len(final_features_val)}")
print(f"Shape train : {df_train_pivot.shape}")
print(f"Shape val   : {df_val_pivot.shape}")

# Vérification que OS_YEARS et OS_STATUS sont bien présents dans les données d'entraînement
print(f"OS_YEARS dans train : {'OS_YEARS' in df_train_pivot.columns}")
print(f"OS_STATUS dans train : {'OS_STATUS' in df_train_pivot.columns}")



Nombre de gènes utilisés (top 150) : 124
Nombre de colonnes finales train (avec targets) : 204
Nombre de colonnes finales val (sans targets) : 202
Shape train : (3173, 204)
Shape val   : (1193, 202)
OS_YEARS dans train : True
OS_STATUS dans train : True


# Molecular Risk Score 

We first build a **regularized Cox proportional hazards model** using all gene mutation features from the training set. The model coefficients are then combined into a single **molecular risk score** for each patient (linear predictor / log-hazard). Regularization is used to reduce overfitting and stabilize coefficient estimates.

To make the score clinically interpretable, we divide patients into **quartiles** of risk:
- Q1 = lowest risk  
- Q4 = highest risk  

We then plot **Kaplan–Meier survival curves** for these quartile groups. A clear separation of curves indicates that the molecular score successfully stratifies patients according to prognosis.

Finally, we compute the **concordance index (C-index)** for the risk score alone, which measures its discriminative ability. A C-index > 0.5 indicates predictive value (0.5 = random, 1.0 = perfect prediction).


In [99]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.utils import concordance_index


In [100]:
from lifelines import CoxPHFitter

gene_cols = [c for c in df_train_pivot.columns if c.startswith("Gene_")]

cox_cols_train = ['OS_YEARS', 'OS_STATUS'] + gene_cols
cox_df_train = df_train_pivot[cox_cols_train].copy()
cox_df_train['OS_STATUS'] = cox_df_train['OS_STATUS'].astype(int)

# Filter rare / almost-constant genes
gene_mat = cox_df_train[gene_cols]
prevalence = gene_mat.mean(axis=0)

min_prop = 0.01
max_prop = 0.99

valid_genes = prevalence[(prevalence >= min_prop) & (prevalence <= max_prop)].index.tolist()
print(f"Initial number of genes: {len(gene_cols)}")
print(f"Number of genes kept: {len(valid_genes)}")
cox_df_train = cox_df_train[['OS_YEARS', 'OS_STATUS'] + valid_genes]

cph = CoxPHFitter(penalizer=0.1)
cph.fit(cox_df_train, duration_col='OS_YEARS', event_col='OS_STATUS')


Initial number of genes: 124
Number of genes kept: 47


<lifelines.CoxPHFitter: fitted with 3173 total observations, 1573 right-censored observations>

In [101]:
X_train = cox_df_train[valid_genes]
df_train_pivot['RiskScore'] = cph.predict_log_partial_hazard(X_train)


C:\Users\arthr\AppData\Local\Temp\ipykernel_69820\2669662178.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [102]:
features_used = valid_genes 

X_val = df_val_pivot.copy()

missing_cols = set(features_used) - set(X_val.columns)
if missing_cols:
    print("Warning: missing gene columns in prediction set:", missing_cols)
    # set them to 0 (gene never present in this cohort)
    for col in missing_cols:
        X_val[col] = 0.0

X_val = X_val[features_used]

df_val_pivot['RiskScore'] = cph.predict_log_partial_hazard(X_val)


C:\Users\arthr\AppData\Local\Temp\ipykernel_69820\1430241184.py:14: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [103]:
df_train_pivot['RiskQuartile'] = pd.qcut(
    df_train_pivot['RiskScore'],
    q=4,
    labels=['Q1 (lowest)', 'Q2', 'Q3', 'Q4 (highest)']
)

df_val_pivot['RiskQuartile'] = pd.qcut(
    df_val_pivot['RiskScore'],
    q=4,
    labels=['Q1 (lowest)', 'Q2', 'Q3', 'Q4 (highest)']
)


C:\Users\arthr\AppData\Local\Temp\ipykernel_69820\3615651102.py:1: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\arthr\AppData\Local\Temp\ipykernel_69820\3615651102.py:7: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [104]:
from lifelines import KaplanMeierFitter
import plotly.graph_objects as go

kmf = KaplanMeierFitter()

fig = go.Figure()

quartile_order = ['Q1 (lowest)', 'Q2', 'Q3', 'Q4 (highest)']

for q in quartile_order:
    sub = df_train_pivot[df_train_pivot['RiskQuartile'] == q]

    kmf.fit(
        durations=sub['OS_YEARS'],
        event_observed=sub['OS_STATUS'],
        label=q
    )

    sf = kmf.survival_function_

    fig.add_trace(go.Scatter(
        x=sf.index,
        y=sf.iloc[:, 0],   # first (and only) column
        mode='lines',
        name=q
    ))

fig.update_layout(
    xaxis_title="Time (years)",
    yaxis_title="Survival probability",
    legend_title="Risk quartile",
    template="simple_white",
    margin=dict(l=60, r=40, t=20, b=40)
)

fig.show()


### Evaluation of the risk index

In [105]:
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation


scores = k_fold_cross_validation(
    cph,
    cox_df_train[['OS_YEARS','OS_STATUS'] + valid_genes],
    duration_col='OS_YEARS',
    event_col='OS_STATUS',
    k=5,
    scoring_method="concordance_index"
)

print("Mean C-index:", np.mean(scores))
print("Std:", np.std(scores))


Mean C-index: 0.6921342917536805
Std: 0.020814991931316064


In [106]:
cph.summary[['coef']]


,coef
covariate,
Gene_TET2,-0.048504
Gene_ASXL1,0.302685
Gene_SF3B1,-0.141947
Gene_SRSF2,0.206318
Gene_DNMT3A,0.212677
Gene_RUNX1,0.449281
Gene_TP53,1.039290
Gene_STAG2,0.272444
Gene_U2AF1,0.172069


In [107]:
df_train_pivot = pd.get_dummies(
    df_train_pivot,
    columns=['RiskQuartile'],
    drop_first=False
)

df_val_pivot = pd.get_dummies(
    df_val_pivot,
    columns=['RiskQuartile'],
    drop_first=False
)


# Other Feature engineering

In [ ]:
df_train_pivot['cyto_risk_score'] = (
    3 * df_train_pivot['is_monosomal_karyotype'] +
    3 * df_train_pivot['is_complex_karyotype'] +
    2 * df_train_pivot['has_minus7_or_del7q'] +
    2 * df_train_pivot['has_minus5_or_del5q'] +
    1 * df_train_pivot['has_plus8']
)

df_val_pivot['cyto_risk_score'] = (
    3 * df_val_pivot['is_monosomal_karyotype'] +
    3 * df_val_pivot['is_complex_karyotype'] +
    2 * df_val_pivot['has_minus7_or_del7q'] +
    2 * df_val_pivot['has_minus5_or_del5q'] +
    1 * df_val_pivot['has_plus8']
)

df_train_pivot['TP53_complex_interaction'] = df_train_pivot['Gene_TP53'] * df_train_pivot['is_complex_karyotype']
df_train_pivot['ASXL1_minus7_interaction'] = df_train_pivot['Gene_ASXL1'] * df_train_pivot['has_minus7_or_del7q']
df_train_pivot['NPM1_normal_interaction'] = df_train_pivot['Gene_NPM1'] * (1 - df_train_pivot['prop_any_abnormal'])

df_val_pivot['TP53_complex_interaction'] = df_val_pivot['Gene_TP53'] * df_val_pivot['is_complex_karyotype']
df_val_pivot['ASXL1_minus7_interaction'] = df_val_pivot['Gene_ASXL1'] * df_val_pivot['has_minus7_or_del7q']
df_val_pivot['NPM1_normal_interaction'] = df_val_pivot['Gene_NPM1'] * (1 - df_val_pivot['prop_any_abnormal'])

df_train_pivot['high_risk_chr_load'] = (
    df_train_pivot["CHR_5_count"] +
    df_train_pivot['CHR_7_count'] +
    df_train_pivot['CHR_17_count']
)

df_val_pivot['high_risk_chr_load'] = (
    df_val_pivot["CHR_5_count"] +
    df_val_pivot['CHR_7_count'] +
    df_val_pivot['CHR_17_count']
)



In [109]:
df_train_pivot['risk_score_high_genes'] = (
    df_train_pivot['Gene_TP53'] +
    df_train_pivot['Gene_ASXL1'] +
    df_train_pivot['Gene_RUNX1']
)

df_train_pivot['risk_score_favorable_genes'] = df_train_pivot['Gene_NPM1'] + df_train_pivot['Gene_CEBPA']

df_val_pivot['risk_score_high_genes'] = (
    df_val_pivot['Gene_TP53'] +    
    df_val_pivot['Gene_ASXL1'] +
    df_val_pivot['Gene_RUNX1']
)

df_val_pivot['risk_score_favorable_genes'] = df_val_pivot['Gene_NPM1'] + df_val_pivot['Gene_CEBPA']


df_train_pivot['n_splicing_mut'] = df_train_pivot[['Gene_U2AF1','Gene_SRSF2','Gene_SF3B1','Gene_ZRSR2']].sum(axis=1)
df_train_pivot['n_signaling_mut'] = df_train_pivot[['Gene_NRAS','Gene_KRAS','Gene_JAK2','Gene_CBL']].sum(axis=1)

df_val_pivot['n_splicing_mut'] = df_val_pivot[['Gene_U2AF1','Gene_SRSF2','Gene_SF3B1','Gene_ZRSR2']].sum(axis=1)
df_val_pivot['n_signaling_mut'] = df_val_pivot[['Gene_NRAS','Gene_KRAS','Gene_JAK2','Gene_CBL']].sum(axis=1)

df_train_pivot['TP53_VAF_interaction'] = df_train_pivot['Gene_TP53'] * df_train_pivot['VAF_avg']
df_val_pivot['TP53_VAF_interaction'] = df_val_pivot['Gene_TP53'] * df_val_pivot['VAF_avg']


df_train_pivot['ANC_WBC_ratio'] = df_train_pivot['ANC'] / (df_train_pivot['WBC']+1)
df_train_pivot['BLAST_WBC_ratio'] = df_train_pivot['BM_BLAST'] / (df_train_pivot['WBC']+1)

df_val_pivot['ANC_WBC_ratio'] = df_val_pivot['ANC'] / (df_val_pivot['WBC']+1)
df_val_pivot['BLAST_WBC_ratio'] = df_val_pivot['BM_BLAST'] / (df_val_pivot['WBC']+1)







df_train_pivot['major_clone_VAF'] = df_train_pivot['VAF_max']
df_train_pivot['subclonality'] = df_train_pivot['VAF_std'] / (df_train_pivot['VAF_avg']+1e-6)

df_val_pivot['major_clone_VAF'] = df_val_pivot['VAF_max']
df_val_pivot['subclonality'] = df_val_pivot['VAF_std'] / (df_val_pivot['VAF_avg']+1e-6)


df_train_pivot['karyo_score_clinical'] = (
    3 * df_train_pivot['is_monosomal_karyotype'] +
    2 * df_train_pivot['is_complex_karyotype'] +
    2 * df_train_pivot['has_minus7_or_del7q'] +
    1 * df_train_pivot['has_plus8']
)

df_val_pivot['karyo_score_clinical'] = (
    3 * df_val_pivot['is_monosomal_karyotype'] +
    2 * df_val_pivot['is_complex_karyotype'] +
    2 * df_val_pivot['has_minus7_or_del7q'] +
    1 * df_val_pivot['has_plus8']
)



In [110]:
import numpy as np
import pandas as pd

def compute_vaf_entropy(df_mut):
    """
    df_mut doit contenir au minimum :
        - 'ID'  : identifiant patient
        - 'VAF' : fraction variant allele
    Retourne un dataframe avec une seule ligne par patient :
        ID | vaf_entropy
    """

    # Fonction locale de Shannon entropy
    def entropy_from_vaf(vaf_list):
        vaf_arr = np.array(vaf_list)
        
        # Normalisation → proportions p_i
        p = vaf_arr / vaf_arr.sum()

        # Somme seulement sur p_i > 0 sinon log pose pb.
        p = p[p > 0]

        return -np.sum(p * np.log(p))

    entropy_per_patient = (
        df_mut.groupby('ID')['VAF']
              .apply(entropy_from_vaf)
              .reset_index()
              .rename(columns={'VAF': 'vaf_entropy'})
    )

    return entropy_per_patient


entropy_train = compute_vaf_entropy(maf_df)
entropy_eval = compute_vaf_entropy(maf_eval)
    
df_train_pivot = df_train_pivot.merge(entropy_train, on='ID', how='left')
df_val_pivot = df_val_pivot.merge(entropy_eval, on='ID', how='left')

df_train_pivot["vaf_entropy"] = df_train_pivot["vaf_entropy"].fillna(0)
df_val_pivot["vaf_entropy"] = df_val_pivot["vaf_entropy"].fillna(0)


In [111]:
df_train_pivot["vaf_entropy"] = df_train_pivot["vaf_entropy"].fillna(0)
df_val_pivot["vaf_entropy"] = df_val_pivot["vaf_entropy"].fillna(0)

In [112]:
def safe_ratio(num, den):
    num = num.astype(float)
    den = den.astype(float)
    res = num / den
    res[~np.isfinite(res)] = np.nan   # remplace inf, -inf, nan par NaN
    return res

# 1) Remplacer les -1 par NaN dans HB
for df in [df_train_pivot, df_val_pivot]:
    df.loc[df["HB"] == -1, "HB"] = np.nan

# 2) Calculer la médiane de HB sur le train
hb_median = df_train_pivot["HB"].median()

# 3) Imputer les NaN de HB avec cette médiane (même valeur pour train et eval)
df_train_pivot["HB"] = df_train_pivot["HB"].fillna(hb_median)
df_val_pivot["HB"]  = df_val_pivot["HB"].fillna(hb_median)

# 4) Calculer le ratio PLT_HB_ratio
for df in [df_train_pivot, df_val_pivot]:
    df["PLT_HB_ratio"] = safe_ratio(df["PLT"], df["HB"] + 1)

In [113]:
for df_ in [df_train_pivot, df_val_pivot]:
    for col in ['WBC', 'ANC', 'PLT', 'MONOCYTES', 'BM_BLAST', 'Nmut', 'DEPTH_avg']:
        if col in df_.columns:
            df_[f"log1p_{col}"] = np.log1p(df_[col].clip(lower=0))


In [114]:
mutation_cols = [c for c in df_train_pivot.columns if c.startswith("EFFECT_") and c.endswith("_count")]

low_freq_cols = []
for c in mutation_cols:
    if (df_train_pivot[c] > 0).mean() < 0.01:   # présent chez <1% des patients
        low_freq_cols.append(c)

df_train_pivot = df_train_pivot.drop(columns=low_freq_cols)
df_val_pivot   = df_val_pivot.drop(columns=[c for c in low_freq_cols if c in df_val_pivot.columns])


In [115]:
print("EFFECT_LOF_count" in df_train_pivot.columns)
print([c for c in df_train_pivot.columns if "EFFECT" in c])


False
['EFFECT_FV_count', 'EFFECT_LOF_ratio', 'EFFECT_NS_count', 'EFFECT_SG_count', 'EFFECT_nunique']


In [116]:
for df_ in [df_train_pivot, df_val_pivot]:
    df_["mutation_burden_score"] = (
        0.5 * df_["Nmut"] +
        1.5 * df_["EFFECT_LOF_ratio"] +
        1.0 * df_["vaf_entropy"]
    )


In [117]:
for df_ in [df_train_pivot, df_val_pivot]:
    for col in ["n_events", "n_chromosomes_altered", "n_monosomies_total", "n_trisomies_total"]:
        if col in df_.columns:
            df_[f"log1p_{col}"] = np.log1p(df_[col])


In [122]:
df_train_pivot.isna().sum()[df_train_pivot.isna().sum() > 0]

high_risk_chr_load    1980
dtype: int64

In [118]:
# ======================================================================
# Sauvegarde
# ======================================================================
df_train_pivot.to_csv('../../data/train_enhanced.csv', index=False)
df_val_pivot.to_csv('../../data/eval_enhanced.csv',  index=False)